# Data Cleaning

The files we downloaded from kaggle include in game stats for all seasons from 2003 up to 2020. Our predcitions are going to be based off the season average stats of the two relevant teams in each game. In order to get this we have to calculate the averages from all the games they played up to that point in the season, and make a new dataframe for the specific game with averages as input.

We then need to make a target variable. The difficulty with this dataset is, we would like to predict whether the favorite team (based on Ken Pom rankings) won the game for betting purposes, and this dataset does not include a team raking. We had to use another dataset from kaggle which includes the team rankings, check if the winning team in the game was the favorite, and then make a binary column of Favorite Win. 

Ken Pomeroy, the creator of KenPom, uses analytical measures to rank each team in college basketball. He is one of the most popular and well-known names in basketball analytics and has been at the forefront of the progression mainstream acceptance of advanced stats.

Once we have a target column, we can create a dataframe with full seasons worth of games with a target column, and train our model on it.

In [1]:
import pandas as pd
import numpy as np
import time

In [2]:
# Importing a regular season dataframe, a postseason dataframe, a rankings dataframe, 
# and a dataframe with Ken Pom ranking from the rankings dataframe. 

regularseason = pd.read_csv('../Data/regularseason22.csv')
postseason = pd.read_csv('../Kaggle_Datasets/MNCAATourneyDetailedResults.csv')
rankings = pd.read_csv('../Kaggle_Datasets/MMasseyOrdinals.csv')
kenpom = pd.read_csv('../Data/KenPom20212109.csv')
team_ids = pd.read_csv('../Data/TeamNames.csv')

In [3]:
team_ids_dict = dict(zip(team_ids.KenPom,team_ids.TeamID))
kenpom = kenpom.rename(columns = {'RankAdjEM' : 'OrdinalRank', 'TeamName': 'TeamID'})
kenpom.TeamID = kenpom['TeamID'].replace(team_ids_dict)

In [4]:
regularseason.drop('Unnamed: 0', axis = 1, inplace = True)

In [5]:
regularseason.WTeamID = regularseason.WTeamID.astype(str).str.extract('(\d+)').astype(float, errors = 'ignore')
regularseason.LTeamID = regularseason.LTeamID.astype(str).str.extract('(\d+)').astype(float, errors = 'ignore')
regularseason.dropna(inplace = True)

In [6]:
regularseason.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1292 entries, 0 to 1668
Data columns (total 34 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   Season   1292 non-null   int64  
 1   DayNum   1292 non-null   int64  
 2   WTeamID  1292 non-null   float64
 3   WScore   1292 non-null   int64  
 4   LTeamID  1292 non-null   float64
 5   LScore   1292 non-null   int64  
 6   WLoc     1292 non-null   object 
 7   NumOT    1292 non-null   int64  
 8   WFGM     1292 non-null   int64  
 9   WFGA     1292 non-null   int64  
 10  WFGM3    1292 non-null   int64  
 11  WFGA3    1292 non-null   int64  
 12  WFTM     1292 non-null   int64  
 13  WFTA     1292 non-null   int64  
 14  WOR      1292 non-null   int64  
 15  WDR      1292 non-null   int64  
 16  WAst     1292 non-null   int64  
 17  WTO      1292 non-null   int64  
 18  WStl     1292 non-null   int64  
 19  WBlk     1292 non-null   int64  
 20  WPF      1292 non-null   int64  
 21  LFGM     1292 

We want to have some more advanced stats to use for prediction. We create them here with a few calculations from the regular stats.

In [7]:
def add_rate_cols(df):
    # Calculates the number of possesions for each team in each game. The number of possessions
    # has a large effect on teams averages and counting stats. Knowing this tells us how
    # efficient a team is different factors of a game.
    df['possessions'] = .5 * (df['FGA'] + (.475 * df['FTA']) - df['OR'] + df['TO'])
    
    # Effective Field Goal Percentage (eFG) gives extra value to 3-point shots. Each 3-pointer made
    # is given the weight of 1.5 2-pointers made.
    df['eFG'] = (df['FGM'] + (.5 * df['FGM3'])) / df['FGA']
    
    # True Shooting Percentage (TS%) gives weight to the 3-point shot as well as free throws made.
    # This stat takes in account all types of scoring to determine a players shooting percentage.
    df['TS%'] = df['Score'] / (2 * (df['FGA'] + .475 * df['FTA']))
    
    # Free Throw Rate (FTr) is a simple ratio of Free Throws to Field Goals Attempted. It identifies
    # a team's abiilty to draw fouls and get to the free throw line where expected value is high.
    df['FTr'] = df['FTA'] / df['FGA']
    
    # Three Point Attempt Rate (3PAr) shows how often a player shoots 3-pointers compared to other
    # types of shots. It identifies players who attack the basket and draw fouls and players who are
    # likely to stand outside and shoot.
    df['3PAr'] = df['FGA3'] / (df['FGA'] + (.475*df['FTA']))
    
    # Offensive Rebound Percentage (OR%) is the number of offensive rebounds a team gets compared
    # to the total number of rebounds available while they were on offense.
    df['OR%'] = df['OR'] / (df['OR'] + df['Opp_DR'])
    
    # Defensive Rebound Percentage (DR%) is the number of defensive rebounds a team gets compared
    # to the total number of rebounds available while they were on defense.
    df['DR%'] = df['DR'] / (df['DR'] + df['Opp_OR'])
    
    # Rebound Percentage (REB%) is the percentage of rebounds from all available rebounds a team
    # gets in a game.
    df['REB%'] = (df['OR'] + df['DR']) / (df['OR'] + df['Opp_DR'] + df['Opp_OR'] + df['DR'])
    
    # Total Rebounds (TR) the number of rebounds a team gets in a game.
    df['TR'] = (df['OR'] + df['DR'])
    
    # Assist to Turnover Ratio (ATOr) shows how well a team takes care of the ball. The higher the
    # the number the better a team is at passing the ball and playing together.
    df['ATOr'] = df['Ast'] / df['TO']
    
    # Assist Percentage (Ast%) is the number of assists a team has compared to baskets scored. The
    # higher this percentage the better the team is at playing together and creating baskets for
    # one another.
    df['Ast%'] = df['Ast'] / df['FGM']
    
    # Steal Percentage (Stl%) is the rate at which a team gets steals during other teams possessions.
    df['Stl%'] = df['Stl'] / (df['Opp_FGA'] + .475*df['Opp_FTA'] + df['Opp_TO'])
    
    # Block Percentage (Blk%) is the rate at which a team blocks shots during other teams possessions.
    df['Blk%'] = df['Blk'] / (df['Opp_FGA'])
    
    # Turnover Percentage (TO_r) is the rate at which a team turnovers the ball over during their own
    # possessions.
    df['TO_r'] = df['TO'] / (df['FGA'] + .475*df['FTA'] + df['TO'])
    
    # Opponent Effective Field Goal Percentage (Opp_eFG) is a defensive measure that shows the Effective
    # Field Goal Percentage of a team's opponents.
    df['Opp_eFG'] = (df['Opp_FGM'] + (.5 * df['Opp_FGM3'])) / df['Opp_FGA']
    
    # Opponent True Shooting Percentage (Opp_TSpct) is a defensive measure that shows the True Shooting
    # Percentage of a team's opponents.
    df['Opp_TSpct'] = df['Opp_Score'] / (2 * (df['Opp_FGA'] + .475 * df['Opp_FTA']))
    
    # Opponent Free Throw Rate (Opp_FTr) is a defensive measure that shows how often a team fouls their
    # opponents and allows them to shoot free throws.
    df['Opp_FTr'] = df['Opp_FTA'] / df['Opp_FGA']
    
    # Opponent Three Point Rate (Opp_3Pr) is a defensive rate measure of how many 3-pointers an
    # opponent makes per basket scored.
    df['Opp_3Pr'] = df['Opp_FGA3'] / (df['Opp_FGA'] + (.475*df['Opp_FTA']))
    
    # Opponent Total Rebounds (Opp_TR) is the number of rebounds a team's opponent gets in a game.
    df['Opp_TR'] = (df['Opp_OR'] + df['Opp_DR'])
    
    # Opponent Assist to Turnover Ratio (Opp_ATOr) is a defensive rate measure of an opponents
    # assists per turnover. This is a good measure of defensive aggressiveness and the ability to make
    # opponents work hard for their shots.
    df['Opp_ATOr'] = df['Opp_Ast'] / df['Opp_TO']
    
    # Opponent Assist Percentage (Opp_Ast_%) is the percentage of opponent's baskets that are assisted.
    df['Opp_Ast_%'] = df['Opp_Ast'] / df['Opp_FGM']
    
    # Opponent Steal Percentage (Opp_Stl_%) is the percentage possesions that end with the opponent getting
    # a steal.
    df['Opp_Stl_%'] = df['Opp_Stl'] / (df['FGA'] + .475*df['FTA'] + df['TO'])
    
    # Opponent Block Percentage (Opp_Blk_%) is the percentage of opponent's possessions that end in a
    # block.
    df['Opp_Blk_%'] = df['Opp_Blk'] / (df['FGA'])
    
    # Opponent Turnover Rate (Opp_TO_r) is the percentage of opponent's possesions that end in a 
    # turnover.
    df['Opp_TO_r'] = df['Opp_TO'] / (df['Opp_FGA'] + .475*df['Opp_FTA'] + df['Opp_TO'])

    return df

This function will take all the in-game stats of a teams season, and put it into one dataframe. Then (if the paramter average is set to True), put that into a single row with the teams averages.

In [1]:
# This function is used to get the stats for a team in the regular season. The parameters are:
# the team id number, the season, the day number of the season, and whether you want to see
# each individual game or the season averages.

def team_regular_season(team_id,Season = 2022, DayNum = 130, average = True):
    
    # create a dataframe that has all the wins/losses of a team during the season up to the given day number.
    teamwins = regularseason[(regularseason['WTeamID'] == team_id) & (regularseason['Season'] == Season) & (regularseason['DayNum'] < DayNum)] 
    teamlosses = regularseason[(regularseason['LTeamID'] == team_id) & (regularseason['Season'] == Season) & (regularseason['DayNum'] < DayNum)]
    
    # split the dataframes to only have the offensive statistics of the given team
    teamwinsoffense = teamwins.filter(like = 'W')    
    teamlossesoffense = teamlosses.filter(like = 'L')

    # split the dataframes to only have the defensive statistics of the given team
    teamwinsoffense.columns = teamwinsoffense.columns.str.lstrip('W')
    teamlossesoffense.columns = teamlossesoffense.columns.str.lstrip('L')

    # concat the two offensive dataframes
    teamoffense = pd.concat([teamwinsoffense, teamlossesoffense])

    # create oppposing team statistics from the given games
    teamwinsopponents = teamwins.filter(like = 'L')
    teamlossesopponents = teamlosses.filter(like = 'W')

    # rename the columns from the opponent statistics dataframes
    teamwinsopponents.columns = teamwinsopponents.columns.str.lstrip('L')
    teamlossesopponents.columns = teamlossesopponents.columns.str.lstrip('W')

    # concat the dataframes from opponent wins and losses
    teamopponents = pd.concat([teamwinsopponents, teamlossesopponents])
    
    # concat the team stats and the opponent stats to create a full picture of the team's season
    teamtotal = pd.concat([teamoffense, teamopponents.add_prefix('Opp_')], axis = 1)
    
    teamtotal.drop('Opp_TeamID', axis = 1, inplace = True)
    
    # return either the season averages or the game by game statistics
    if average == True:
        teamtotal = teamtotal.groupby('TeamID').mean().reset_index()
        add_rate_cols(teamtotal)
    
    if average == False:
        add_rate_cols(teamtotal)
        
    return teamtotal

SyntaxError: invalid syntax (<ipython-input-1-9d09c61c56cc>, line 5)

In [9]:
# Here's an example of a team_regular_season output.
Gonzaga = team_regular_season(1211, 2022, 20, False)
Gonzaga

,TeamID,Score,Loc,FGM,FGA,FGM3,FGA3,FTM,FTA,OR,...,Opp_eFG,Opp_TSpct,Opp_FTr,Opp_3Pr,Opp_TR,Opp_ATOr,Opp_Ast_%,Opp_Stl_%,Opp_Blk_%,Opp_TO_r
0,1211.0,97,Home,37,68,6,21,17,24,14,...,0.415385,0.420140,0.323077,0.293431,32,0.600000,0.521739,0.052966,0.014706,0.210582
334,1211.0,86,Home,32,59,7,19,15,17,7,...,0.516949,0.555556,0.271186,0.360360,31,1.222222,0.423077,0.054738,0.016949,0.119048
445,1211.0,84,Home,34,58,7,19,9,12,5,...,0.427419,0.436280,0.112903,0.168389,25,0.909091,0.400000,0.117340,0.017241,0.144121
675,1211.0,92,Home,32,60,11,23,17,21,14,...,0.424528,0.447628,0.113208,0.447628,17,0.533333,0.380952,0.036155,0.033333,0.211715


Now we get the end of year rankings of a team for a specific year with this function.

In [10]:
def get_KenPom(team_id, Year):
    rank = kenpom[(kenpom['TeamID'] == team_id) & (kenpom['Season'] == Year)]
    rank.reset_index(inplace = True)
    return rank.OrdinalRank

get_KenPom(1181, 2022)

0    7
Name: OrdinalRank, dtype: int64

This function will take in a specified matchup and output a game with the two teams season stats and whether the favorite team one.

In [11]:
def single_game(team1_id, team2_id, Year = 2020, DayNum = 132):
    
    # from the Team IDs get a dataframe with the average seasonlong team statistics.
    team1 = team_regular_season(team1_id,Year,DayNum)
    team2 = team_regular_season(team2_id,Year,DayNum)
    
    # get the Ken Pom ranking for each team
    team1Pom = get_KenPom(team1_id, Year)
    team2Pom = get_KenPom(team2_id, Year)
    
    # due to the structure of the original dataframe, the first team listed is always the winner.
    # we check if the favorite won by comparing the rankings of the two teams.
    fav_win = pd.DataFrame(team1Pom.lt(team2Pom))
    
    # concatenate the dataframes and return the single game pre-game data.
    game = pd.concat([team1.add_prefix('W_'), team2.add_prefix('L_'), fav_win], axis = 1)
    game.rename(columns={game.columns[106]: 'fav_win'}, inplace = True)
    
    return game

This code shows how the stats from the function "single_game" update throughout the season. In this 
hypothetical example Duke plays North Carolina during the 2020 season on four separate occasions. On Day 0
there are no stats available so all the values are 'Nan'. On Day 2, only Duke has played a game, they have
stats available, however, North Carolina has not, so their values are all 'Nan'. On Day 50, both teams have
played and have available stats. On Day 100, both teams have continued deep into their seasons and have
updated statistics from the first 100 days of the year.

In [12]:
duke_unc = single_game(1314, 1181, 2022, 0)
duke_unc

,W_TeamID,W_Score,W_FGM,W_FGA,W_FGM3,W_FGA3,W_FTM,W_FTA,W_OR,W_DR,...,L_Opp_TSpct,L_Opp_FTr,L_Opp_3Pr,L_Opp_TR,L_Opp_ATOr,L_Opp_Ast_%,L_Opp_Stl_%,L_Opp_Blk_%,L_Opp_TO_r,fav_win
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False


In [13]:
duke_unc = single_game(1181, 1314, 2022, 2)
duke_unc

,W_TeamID,W_Score,W_FGM,W_FGA,W_FGM3,W_FGA3,W_FTM,W_FTA,W_OR,W_DR,...,L_Opp_TSpct,L_Opp_FTr,L_Opp_3Pr,L_Opp_TR,L_Opp_ATOr,L_Opp_Ast_%,L_Opp_Stl_%,L_Opp_Blk_%,L_Opp_TO_r,fav_win
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True


In [14]:
duke_unc = single_game(1181, 1314, 2022, 25)
duke_unc

,W_TeamID,W_Score,W_FGM,W_FGA,W_FGM3,W_FGA3,W_FTM,W_FTA,W_OR,W_DR,...,L_Opp_TSpct,L_Opp_FTr,L_Opp_3Pr,L_Opp_TR,L_Opp_ATOr,L_Opp_Ast_%,L_Opp_Stl_%,L_Opp_Blk_%,L_Opp_TO_r,fav_win
0,1181.0,85.833333,32.5,66.166667,8.166667,24.333333,12.666667,16.166667,11.0,27.833333,...,0.526023,0.232673,0.343252,28.666667,1.865385,0.541899,0.096361,0.064789,0.103865,True


In [15]:
duke_unc = single_game(1181, 1314, 2022, 30)
duke_unc

,W_TeamID,W_Score,W_FGM,W_FGA,W_FGM3,W_FGA3,W_FTM,W_FTA,W_OR,W_DR,...,L_Opp_TSpct,L_Opp_FTr,L_Opp_3Pr,L_Opp_TR,L_Opp_ATOr,L_Opp_Ast_%,L_Opp_Stl_%,L_Opp_Blk_%,L_Opp_TO_r,fav_win
0,1181.0,83.125,31.375,66.125,7.5,22.875,12.875,17.25,10.0,26.625,...,0.526023,0.232673,0.343252,28.666667,1.865385,0.541899,0.096361,0.064789,0.103865,True




We can now write a function to create a full seasons worth of games using the single_game function to concatenate all of the single games of a season into one DataFrame. 

In [16]:
def create_season_df(df, season):
    
    # create a DataFrame for the season we are looking for
    season_df = df[(df['Season'] == season) & (df['DayNum'] >= 12)]
    season_df.reset_index(inplace = True)
    
    # create a list with the two teams, winning team always first, the season, and the day number of the game
    matchups = list(zip(season_df.WTeamID, season_df.LTeamID, season_df.Season, season_df.DayNum))
    
    season_games = []
    
    # iterate through the list we created and create each single game matchup throughout the season
    for i in range(len(matchups)):
        game = single_game(matchups[i][0], matchups[i][1], Year = matchups[i][2], DayNum = matchups[i][3])
        season_games.append(game)
    
    # from the season games list concatenate all the outputted DataFrames and insert the location
    # of the winner (home, away, neutral)
    df = pd.concat(season_games, axis = 0).dropna()
    df.reset_index(inplace = True, drop = True)
    df.insert(0,"w_loc", season_df.WLoc)
    
    return df

Now we can simply run this last function on each season and each postseason. With that, our data cleaning is complete.

In [17]:
# ###7.5 seconds to run
# start_time = time.time()

regularseason22 = create_season_df(regularseason, 2022)

# end_time = time.time()
# print('Execution time = %.6f seconds' % (end_time-start_time))

In [24]:
regularseason22.to_csv('../data/regularseason22.csv')

In [19]:
# postseason13 = create_season_df(postseason, 2013)
# postseason14 = create_season_df(postseason, 2014)
# postseason15 = create_season_df(postseason, 2015)
# postseason16 = create_season_df(postseason, 2016)
# postseason17 = create_season_df(postseason, 2017)
# postseason18 = create_season_df(postseason, 2018)
# postseason19 = create_season_df(postseason, 2019)

In [20]:
# start_time = time.time()

#regularseason12 = create_season_df(regularseason, 12)

# end_time = time.time()
# print('Execution time = %.6f seconds' % (end_time-start_time))

ValueError: No objects to concatenate

In [ ]:
# regularseason13 = create_season_df(regularseason, 2013)

In [ ]:
# regularseason14 = create_season_df(regularseason, 2014)

In [ ]:
# regularseason15 = create_season_df(regularseason, 2015)

In [ ]:
# regularseason16 = create_season_df(regularseason, 2016)

In [ ]:
# regularseason17 = create_season_df(regularseason, 2017)

In [ ]:
# regularseason18 = create_season_df(regularseason, 2018)

In [ ]:
# regularseason19 = create_season_df(regularseason, 2019)

In [ ]:
# regularseason20 = create_season_df(regularseason, 2020)

Because this is so computationally expensive we can save these data frames as csv files so we never need to create these dataframes again. So We'll comment out those lines of code.

In [ ]:
# postseason12.to_csv('./data/postseason12.csv')
# postseason13.to_csv('./data/postseason13.csv')
# postseason14.to_csv('./data/postseason14.csv')
# postseason15.to_csv('./data/postseason15.csv')
# postseason16.to_csv('./data/postseason16.csv')
# postseason17.to_csv('./data/postseason17.csv')
# postseason18.to_csv('./data/postseason18.csv')
# postseason19.to_csv('./data/postseason19.csv')

In [ ]:
# regularseason12.to_csv('./data/regularseason12.csv')
# regularseason13.to_csv('./data/regularseason13.csv')
# regularseason14.to_csv('./data/regularseason14.csv')
# regularseason15.to_csv('./data/regularseason15.csv')